In [3]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from datetime import datetime
import glob
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
path = '/media/onno/Algemeen/Thesis/'
# file_gfs = 'gefsrf2_env_wledit2000-10000_latavg_v300_control0-252h_6hourly_2x2_dec84-nov19.nc'
# file_era = 'era51_env_wledit2000-10000_latavg_v300_79-19_6hourly_anom_from_smoothed04_clim_smoothed.nc'
file_gfs = 'gefsrf2_phasevel_wledit2000-10000_latavg_v300_envgt15_control0-252h_6hourly_2x2_dec84-nov19_setvrange_-100to100.nc'
file_era = 'era51_phasevel_wledit2000-10000_latavg_v300_envgt15_79-19_6hourly_anom_from_smoothed04_clim_setvrange_-100to100.nc'
gfs = xr.open_dataset(path+file_gfs,decode_times=False)
era = xr.open_dataset(path+file_era,decode_times=False)
#Convert dates to standard datetime values
gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
era['time']=my_tools.convert_date_era_r(era.time.values)


In [13]:
gfs_nh = gfs.sel(lat=slice(90,0),lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])
era_nh = era.sel(lat=slice(90,0))

Calculation for each season

In [14]:
#Categorize each month for season
seasons = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}

#Loop through season
for key in seasons:
    #loop through lead days
    for i in range(11):
        #Select days for which GEFS data is present and make datasets equally long
        gfs_nh_d = gfs_nh.sel(time=[i for i in pd.date_range('1984-12-1','2019-11-{}'.format(30-i))],lead=i*24)
        era_nh_d = era_nh.sel(time=[i for i in pd.date_range('1984-12-{}'.format(1+i),'2019-11-30')])        
        #Make boolean array for each day that is present in season        
        idxz = np.isin(gfs_nh_d.time.dt.month,seasons[key])
        gfs_season = gfs_nh_d.v[idxz] ; era_season = era_nh_d.v[idxz]
        #Check if dates are correct
        assert(np.all(np.isin([(j-pd.Timedelta(i,'d')).month for j in era_season.time.values],seasons[key])))
        assert(np.all(np.isin([pd.to_datetime(j).month for j in gfs_season.time.values],seasons[key])))
        gfs_season_mean = gfs_season.mean(dim='time')
        era_season_mean = era_season.mean(dim='time')
        diff = gfs_season_mean - era_season_mean
        diff.to_netcdf('/media/onno/Algemeen/Thesis/GFS_mean_season/GFS_mean_error_phasespeed_{}_day_{:02d}.nc'.format(key,i))


In [5]:
filez = glob.glob('/media/onno/Algemeen/Thesis/GFS_mean/*envelope*.nc')
for i, file in enumerate(filez):
    ds = xr.open_dataset(file,decode_times=False).squeeze()
    fig = plt.figure()
    ax = plt.subplot(1,1,1,projection=ccrs.Orthographic(0, 90))
    ax.coastlines()
    im = ax.imshow(ds.v,transform=ccrs.PlateCarree(),
                   vmin=-2.5,vmax=2.5,cmap='seismic')
    cbar = fig.colorbar(im)
    cbar.set_label('E (m/s)', rotation=90)
    fig.suptitle('GEFS mean RWP envelope error day {}'.format(i+1))
    fig.savefig('/media/onno/Algemeen/Thesis/GFS_mean/GEFS_forecast_envelope_error_day_{:02d}'.format(i))
    plt.close(fig)